# Regression Week 5: LASSO Assignment 1

In this assignment, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:

- Run LASSO with different L1 penalties.
- Choose best L1 penalty using a validation set.
- Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second assignment, you will implement your own LASSO solver, using coordinate descent.

If you are using scikit-learn with Pandas:

The instructions may apply to other tools, but the set of parameters are specific to scikit-learn.

## 0. Load the sales dataset using Pandas:

In [1]:
import pandas as pd

dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

## 1. Create new features by performing following transformation on inputs:

In [3]:
from math import log, sqrt

sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

- Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.

- On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

## 2. Using the entire house dataset, learn regression weights using an L1 penalty of 5e2. 

Make sure to add "normalize=True" when creating the Lasso object. Refer to the following code snippet for the list of features.



In [22]:
from sklearn import linear_model  # using scikit-learn

all_features = [
    'bedrooms',
    'bedrooms_square',
    'bathrooms',
    'sqft_living',
    'sqft_living_sqrt',
    'sqft_lot',
    'sqft_lot_sqrt',
    'floors',
    'floors_square',
    'waterfront',
    'view',
    'condition',
    'grade',
    'sqft_above',
    'sqft_basement',
    'yr_built', 
    'yr_renovated'
]

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

## 3. Quiz Question: Which features have been chosen by LASSO, i.e. which features were assigned nonzero weights?

In [23]:
import numpy as np
print(np.array(all_features)[model_all.coef_ != 0])

['sqft_living' 'view' 'grade']


## 4. To find a good L1 penalty, we will explore multiple values using a validation set. 

Let us do three way split into train, validation, and test sets. Download the provided csv files containing training, validation and test sets.

In [25]:
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)

testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

## 5. Now for each l1_penalty in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type np.logspace(1, 7, num=13).)

- Learn a model on TRAINING data using the specified l1_penalty. Make sure to specify normalize=True in the constructor:

In [31]:
rss = lambda y_hat, y: np.dot((y - y_hat).T, (y-y_hat))
from sklearn.metrics import mean_squared_error

costs = []

for l1_penalty in np.logspace(1,7,num=13):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price'])
    
    predictions = model.predict(validation[all_features])
    print(l1_penalty, mean_squared_error(validation['price'], predictions) * len(validation['price']))
#     print(mean_srss(predictions, validation['price']))



10.0 3.982133273e+14
31.6227766017 3.99041900253e+14
100.0 4.29791604073e+14
316.227766017 4.63739831045e+14
1000.0 6.45898733634e+14
3162.27766017 1.22250685943e+15
10000.0 1.22250685943e+15
31622.7766017 1.22250685943e+15
100000.0 1.22250685943e+15
316227.766017 1.22250685943e+15
1000000.0 1.22250685943e+15
3162277.66017 1.22250685943e+15
10000000.0 1.22250685943e+15


Compute the RSS on VALIDATION for the current model (print or save the RSS)
Report which L1 penalty produced the lower RSS on VALIDATION.

## 6. Quiz Question: Which was the best value for the l1_penalty, i.e. which value of l1_penalty produced the lowest RSS on V

## 7. Now that you have selected an L1 penalty, compute the RSS on TEST data for the model with the best L1 penalty.

## 8. Quiz Question: Using the best L1 penalty, how many nonzero weights do you have? Count the number of nonzero coefficients first, and add 1 if the intercept is also nonzero. A succinct way to do this is

In [34]:
model = linear_model.Lasso(alpha=10, normalize=True)
model.fit(training[all_features], training['price'])
np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)

15

## 9. What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

You are going to implement a simple, two phase procedure to achieve this goal:

Explore a large range of ‘l1_penalty’ values to find a narrow region of ‘l1_penalty’ values where models are likely to have the desired number of non-zero weights.
Further explore the narrow region you found to find a good value for ‘l1_penalty’ that achieves the desired sparsity. Here, we will again use a validation set to choose the best value for ‘l1_penalty’.
10. Assign 7 to the variable ‘max_nonzeros’.

In [36]:
for l1_penalty in np.logspace(1,4,num=20):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price'])
    
    print(l1_penalty, np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_))

10.0 15
14.3844988829 15
20.6913808111 15
29.7635144163 15
42.8133239872 13
61.5848211066 12
88.586679041 11
127.42749857 10
183.298071083 7
263.665089873 6
379.269019073 6
545.559478117 6
784.759970351 5
1128.83789168 3
1623.77673919 3
2335.72146909 2
3359.81828628 1
4832.93023857 1
6951.92796178 1
10000.0 1


In [43]:
for l1_penalty in np.linspace(127.42749857,263.665089873,num=20):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price'])
    
    predictions = model.predict(validation[all_features])
    error = mean_squared_error(validation['price'], predictions) * len(validation['price'])
    non0 = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    if (non0 == 7): print(l1_penalty, non0, error)


156.109096739 7 4.40037365263e+14
163.279496281 7 4.40777489642e+14
170.449895824 7 4.4156669809e+14
177.620295366 7 4.42406413189e+14
184.790694908 7 4.43296716874e+14
191.96109445 7 4.44239780526e+14
199.131493993 7 4.45230739843e+14


In [44]:
model = linear_model.Lasso(alpha=156.109096739, normalize=True)
model.fit(training[all_features], training['price'])
print(np.array(all_features)[model.coef_ != 0])

['bathrooms' 'sqft_living' 'waterfront' 'view' 'grade' 'yr_built']
